# Loop 2 Analysis: Investigating Overlap Error

The submission failed with 'Overlapping trees in group 069'. Let's investigate and fix this.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

getcontext().prec = 30

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0')), float(tip_y)),
            (float(top_w / Decimal('2')), float(tier_1_y)),
            (float(top_w / Decimal('4')), float(tier_1_y)),
            (float(mid_w / Decimal('2')), float(tier_2_y)),
            (float(mid_w / Decimal('4')), float(tier_2_y)),
            (float(base_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(base_y)),
            (float(-(base_w / Decimal('2'))), float(base_y)),
            (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
            (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
            (float(-(top_w / Decimal('4'))), float(tier_1_y)),
            (float(-(top_w / Decimal('2'))), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print('ChristmasTree class defined')

In [ ]:
def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def has_overlap(trees):
    if len(trees) <= 1:
        return False, []
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    overlaps = []
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:
                    overlaps.append((i, idx, intersection.area))
    return len(overlaps) > 0, overlaps

print('Helper functions defined')

In [ ]:
# Check candidate_001.csv for overlaps in group 069
df = pd.read_csv('/home/code/submission_candidates/candidate_001.csv')
print(f'Loaded {len(df)} rows')

# Check group 069
trees_69 = load_trees_for_n(df, 69)
print(f'Group 069 has {len(trees_69)} trees')

has_ovlp, overlaps = has_overlap(trees_69)
print(f'Has overlap: {has_ovlp}')
if overlaps:
    print(f'Overlaps found: {overlaps[:10]}')
else:
    print('No overlaps in group 069')

In [ ]:
# Check all groups for overlaps
overlap_groups = []
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    has_ovlp, overlaps = has_overlap(trees)
    if has_ovlp:
        overlap_groups.append((n, len(overlaps)))

print(f'Groups with overlaps: {len(overlap_groups)}')
for n, count in overlap_groups[:20]:
    print(f'  N={n}: {count} overlaps')

In [ ]:
# Compare with baseline (candidate_000.csv)
df_baseline = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')
print('Checking baseline for overlaps...')

overlap_groups_baseline = []
for n in range(1, 201):
    trees = load_trees_for_n(df_baseline, n)
    has_ovlp, overlaps = has_overlap(trees)
    if has_ovlp:
        overlap_groups_baseline.append((n, len(overlaps)))

print(f'Baseline groups with overlaps: {len(overlap_groups_baseline)}')
if overlap_groups_baseline:
    for n, count in overlap_groups_baseline[:10]:
        print(f'  N={n}: {count} overlaps')
else:
    print('Baseline has NO overlaps - this is the valid submission')

In [ ]:
# The issue is clear: candidate_001.csv has overlaps, candidate_000.csv doesn't
# We need to use the baseline (candidate_000.csv) which is valid

# Let's also check the current.csv and submission.csv
import os

print('Checking current.csv...')
if os.path.exists('/home/code/current.csv'):
    df_current = pd.read_csv('/home/code/current.csv')
    overlap_count = 0
    for n in range(1, 201):
        trees = load_trees_for_n(df_current, n)
        has_ovlp, _ = has_overlap(trees)
        if has_ovlp:
            overlap_count += 1
    print(f'current.csv has {overlap_count} groups with overlaps')

print('\nChecking /home/submission/submission.csv...')
if os.path.exists('/home/submission/submission.csv'):
    df_sub = pd.read_csv('/home/submission/submission.csv')
    overlap_count = 0
    for n in range(1, 201):
        trees = load_trees_for_n(df_sub, n)
        has_ovlp, _ = has_overlap(trees)
        if has_ovlp:
            overlap_count += 1
    print(f'/home/submission/submission.csv has {overlap_count} groups with overlaps')